# セッション3-A: Cortex Search の設定

このノートブックでは、チャンク化されたデータに対してCortex Search Serviceを作成し、セマンティック検索を有効にします。

## 前提条件
- セッション2（データ準備）が完了していること
- 統合ビュー（`combined_sustainability_chunks_view`, `combined_global_sustainability_view`）が作成されていること

## このセッションで学ぶこと
1. **Cortex Search Service**の作成
2. 検索テスト
3. 検索結果のカスタマイズ


## 📦 接続設定


In [ ]:
-- 接続情報の設定
USE ROLE ACCOUNTADMIN;
USE DATABASE DEMO_DB;
USE SCHEMA DEMO_SUSTAINABILITY;
USE WAREHOUSE HANDSON_WH;


## 🔍 Cortex Searchとは？

**Cortex Search**は、Snowflake上でセマンティック検索（意味に基づく検索）を実現するマネージドサービスです。

### 主な特徴
- **自動ベクトル化**: テキストを自動的にベクトル（埋め込み）に変換
- **ハイブリッド検索**: セマンティック検索とキーワード検索を組み合わせ
- **自動更新**: ソースデータの変更を自動的に反映（`TARGET_LAG`で設定）
- **フィルタリング**: 属性によるフィルタリングが可能

### 構文

```sql
CREATE CORTEX SEARCH SERVICE サービス名
    ON 検索対象カラム
    ATTRIBUTES 属性カラム1, 属性カラム2, ...
    WAREHOUSE = ウェアハウス名
    TARGET_LAG = '1 hour'
AS (
    SELECT 
        検索対象カラム,
        属性カラム1,
        属性カラム2,
        ...
    FROM ソーステーブル/ビュー
);
```


---
## Step 3-1: 運用機関レポート用 Cortex Search Service

スチュワードシップ原則評価用の検索サービスを作成します。
このサービスは、運用機関のサステナビリティレポートとGPIFスチュワードシップ活動原則の両方を検索対象とします。


In [ ]:
-- スチュワードシップ原則評価用の検索サービス
CREATE OR REPLACE CORTEX SEARCH SERVICE sustainability_report
    ON chunk_text
    ATTRIBUTES relative_path, scoped_file_url, file_name, page_index, chunk_index_on_page
    WAREHOUSE = HANDSON_WH
    TARGET_LAG = '1 hour'
AS (
    SELECT 
        chunk_text,
        relative_path,
        scoped_file_url,
        file_name,
        page_index,
        chunk_index_on_page
    FROM combined_sustainability_chunks_view
);


In [ ]:
-- Cortex Search Service 確認
SHOW CORTEX SEARCH SERVICES;


### 検索テスト

`SEARCH_PREVIEW`関数を使用して、作成した検索サービスをテストします。


In [ ]:
-- 検索テスト: スチュワードシップコードに関する検索
SELECT SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT',
    '{
        "query": "スチュワードシップコードの受入れ",
        "columns": ["chunk_text", "file_name"],
        "limit": 3
    }'
);


In [ ]:
-- 検索テスト: ESGに関する検索
SELECT SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT',
    '{
        "query": "ESG投資方針",
        "columns": ["chunk_text", "file_name", "page_index"],
        "limit": 5
    }'
);


---
## Step 3-2: グローバル年金基金レポート用 Cortex Search Service

グローバル年金分析用の検索サービスを作成します。
このサービスは、GPIFサステナビリティレポートと海外年金基金のレポートを横断的に検索できます。


In [ ]:
-- グローバル年金分析用の検索サービス
CREATE OR REPLACE CORTEX SEARCH SERVICE global_pf_sustainability_report
    ON chunk_text
    ATTRIBUTES relative_path, scoped_file_url, file_name, page_index, chunk_index_on_page, source_report
    WAREHOUSE = HANDSON_WH
    TARGET_LAG = '1 hour'
AS (
    SELECT 
        chunk_text,
        relative_path,
        scoped_file_url,
        file_name,
        page_index,
        chunk_index_on_page,
        source_report
    FROM combined_global_sustainability_view
);


In [ ]:
-- Cortex Search Service 確認
SHOW CORTEX SEARCH SERVICES;


In [ ]:
-- 検索テスト: サステナビリティ戦略に関する検索
SELECT SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'DEMO_DB.DEMO_SUSTAINABILITY.GLOBAL_PF_SUSTAINABILITY_REPORT',
    '{
        "query": "sustainability investment strategy",
        "columns": ["chunk_text", "file_name", "source_report"],
        "limit": 5
    }'
);


---
## 📊 作成したCortex Search Serviceのまとめ

| サービス名 | 用途 | ソースビュー |
|-----------|------|-------------|
| `sustainability_report` | スチュワードシップ原則評価 | `combined_sustainability_chunks_view` |
| `global_pf_sustainability_report` | グローバル年金分析 | `combined_global_sustainability_view` |


---
## ✅ セッション3-A 完了

次のステップ: **セッション3-B: Cortex Agent の設定** (`03_cortex_agent.ipynb`)
